In [1]:
#pip install numpy pandas sklearn joblib

In [2]:
import json # will be needed for saving preprocessing details
import numpy as np # for data manipulation
import pandas as pd # for data manipulation
from sklearn.model_selection import train_test_split # will be used for data split
from sklearn.preprocessing import LabelEncoder # for preprocessing
from sklearn.ensemble import RandomForestClassifier # for training the algorithm
from sklearn.ensemble import ExtraTreesClassifier # for training the algorithm
import joblib # for saving algorithm and preprocessing objects

In [3]:
import statistics
from scipy import stats
from dmba import plotDecisionTree, classificationSummary, regressionSummary
from imblearn.over_sampling import SMOTE
from collections import Counter
from array import array
from sklearn import preprocessing, svm , metrics , tree
from sklearn.model_selection import train_test_split , cross_val_score , GridSearchCV
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

In [4]:
# load dataset
start_ups = pd.read_csv('https://raw.githubusercontent.com/R7amza/project1/R7amza-patch-2/11111.csv', skipinitialspace=True)
start_ups.set_index('Organization_Name', inplace=True)
#x_cols = [c for c in df.columns if c != 'Exit_status']
# set input matrix and target column
#X = df[x_cols]
#y = df['Exit_status']
# show first rows of data
start_ups.info()

<class 'pandas.core.frame.DataFrame'>
Index: 979 entries, Nana to Saudi Re For Cooperative Reinsurance Co.
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Estimated_Revenue_Range  394 non-null    float64
 1   Total_Funding_Amount     210 non-null    float64
 2   Headquarters_Location    979 non-null    object 
 3   Description              979 non-null    object 
 4   Founded_Date             979 non-null    int64  
 5   Number_of_Founders       330 non-null    float64
 6   Funding_Status           349 non-null    object 
 7   Funding_Stage            979 non-null    object 
 8   Number_Funding_Rounds    271 non-null    float64
 9   Number_of_Investors      224 non-null    float64
 10  Industries               962 non-null    object 
 11  Industry_Groups          979 non-null    object 
 12  Sector_Size              979 non-null    int64  
 13  Number_of_Employees      979 non-null    int6

In [5]:
# Replacing missing values
start_ups['Estimated_Revenue_Range'] = start_ups['Estimated_Revenue_Range'].replace({np.NaN :'5500000'})
start_ups['Total_Funding_Amount'] = start_ups['Total_Funding_Amount'].replace({np.NaN :'800000'})
start_ups['Number_of_Founders'] = start_ups['Number_of_Founders'].replace({np.NaN :'2'})
start_ups['Number_Funding_Rounds'] = start_ups['Number_Funding_Rounds'].replace({np.NaN :'1'})
start_ups['Number_of_Investors'] = start_ups['Number_of_Investors'].replace({np.NaN :'1'})
start_ups['Number_of_Apps'] = start_ups['Number_of_Apps'].replace({np.NaN :'1'})
start_ups['Visit_Duration'] = start_ups['Visit_Duration'].replace({np.NaN :'178'})
start_ups['Bounce_Rate'] = start_ups['Bounce_Rate'].replace({np.NaN :'66.56%'})
start_ups['Monthly_Visits'] = start_ups['Monthly_Visits'].replace({np.NaN :'1606'})
start_ups['Monthly_Visits_Growth'] = start_ups['Monthly_Visits_Growth'].replace({np.NaN :'-7.12%'})
start_ups['Exit_status'] = start_ups['Exit_status'].replace({np.NaN :'0'})

In [6]:
# Unifying Headquarters Location names
dict_start = {'Headquarters_Location': {'Riyad ':'Riyadh',
                                        'Riyad':'Riyadh',
                                        'Riyadh ':'Riyadh',
                                        'Jeddah ':'Jeddah',
                                        'Jedda':'Jeddah',
                                        'Sharqiyah':'Ash Sharqiyah',
                                        'Sharqiyah ':'Ash Sharqiyah',
                                        'Ash Sharqiyah ':'Ash Sharqiyah'}}
start_ups.replace(dict_start, inplace=True)

In [7]:
#Classifying Funding Stage in to three stages
dict_start2 = {'Funding_Stage': {'Angel ': 'Early Stage', 
                                 'Angel': 'Early Stage',
                                 'Early Stage ': 'Early Stage',
                                 'Pre-Seed ' : 'Early Stage', 
                                 'Pre-Seed' : 'Early Stage',
                                 'Early Stage ': 'Early Stage',
                                 'Seed ': 'Early Stage', 
                                 'Seed': 'Early Stage', 
                                 'Early Stage ': 'Early Stage',
                                 'Series A ': 'Mid stage', 
                                 'Series A': 'Mid stage',
                                 'Mid stage ': 'Mid stage',
                                 'Series B ': 'Mid Stage',
                                 'Series B': 'Mid stage',
                                 'Sereis B': 'Mid stage',
                                 'Mid stage ': 'Mid stage',
                                 'Series C ': 'Late Stage',
                                 'Series C': 'Late stage',
                                 'Sereis C': 'Late stage',
                                 'Late stage ': 'Late stage'}}
start_ups.replace(dict_start2, inplace=True)

In [8]:
#To replace 'Undisclosed' category by mode.
start_ups['Funding_Stage'] = start_ups['Funding_Stage'].replace({'Undisclosed': np.NaN})
start_ups['Funding_Stage'] = start_ups['Funding_Stage'].fillna(start_ups['Funding_Stage'].value_counts().index[0])

In [9]:
#Unifying Industry groups
dict_start3 = {'Industry_Groups': {'Food Delivery  ': 'Food &Beverage ', 
                                   'Food Delivery ': 'Food &Beverage ',   
                                   'FinTech ' : 'Financial Services', 
                                   'FinTech': 'Financial Services', 
                                   'FintTech' : 'Financial Services',
                                   'Consultancy  ': 'Financial Services',
                                   'Financial Services ' : 'Financial Services',
                                   'Computer Vision ': 'Artificial Intelligence',
                                   'Computer Vision': 'Artificial Intelligence',
                                   'Data Visualization ': 'Artificial Intelligence', 
                                   'Data Visualization': 'Artificial Intelligence', 
                                   'Data Analysis': 'Artificial Intelligence',
                                   'Blockchain ': 'Artificial Intelligence', 
                                   'Blockchain': 'Artificial Intelligence',
                                   'Business Intelligence': 'Artificial Intelligence',
                                   'Artificial Intelligenc' : 'Artificial Intelligence',
                                   'Insurtech' : 'Artificial Intelligence',
                                   'Recruitment ': 'Human Resources',
                                   'Recruitment': 'Human Resources',
                                   'Logistics ':'Travel&Toursim',
                                   'Logistics':'Travel&Toursim',
                                   'Transportation' : 'Travel&Toursim',
                                   'Travel&Toursim ': 'Travel&Toursim',
                                   'Travel&Tourism ': 'Travel&Toursim',
                                   'Food&Beverage ' : 'FMCG',
                                   'Food &Beverage ' : 'FMCG',
                                   'Consumer Services': 'FMCG', 
                                   'Healthtech' : 'HealthTech',
                                   'HealthTech ' : 'HealthTech',
                                   'Financial services ': 'Financial Services',
                                   'Financial Services ': 'Financial Services', 
                                    'Real Estate ': 'Real Estate'}}
start_ups.replace(dict_start3, inplace=True)

In [10]:
start_ups = start_ups.drop(columns = ['Description', 'Funding_Status', 'Industries', 'Downloads_Last_30_Days'])
start_ups_14 = start_ups.query('Founded_Date >2014')
start_ups_14.shape

(451, 17)

In [11]:
# Correcting variables type
start_ups_14['Bounce_Rate'] = start_ups_14['Bounce_Rate'].str.rstrip("%")
start_ups_14['Monthly_Visits_Growth'] = start_ups_14['Monthly_Visits_Growth'].str.rstrip("%")
start_ups_14['Estimated_Revenue_Range'] = start_ups_14['Estimated_Revenue_Range'].astype(float)
start_ups_14['Total_Funding_Amount'] = start_ups_14['Total_Funding_Amount'].astype(float)
start_ups_14['Number_of_Founders'] = start_ups_14['Number_of_Founders'].astype(int)
start_ups_14['Number_Funding_Rounds'] = start_ups_14['Number_Funding_Rounds'].astype(int)
start_ups_14['Number_of_Investors'] = start_ups_14['Number_of_Investors'].astype(int)
start_ups_14['Number_of_Apps'] = start_ups_14['Number_of_Apps'].astype(int)
start_ups_14['Visit_Duration'] = start_ups_14['Visit_Duration'].astype(float)
start_ups_14['Bounce_Rate'] = start_ups_14['Bounce_Rate'].astype(float)
start_ups_14['Monthly_Visits'] = start_ups_14['Monthly_Visits'].astype(float)
start_ups_14['Monthly_Visits_Growth'] = start_ups_14['Monthly_Visits_Growth'].astype(float)
start_ups_14['Sector_Size'] = start_ups_14['Sector_Size'].astype(float)
start_ups_14['Exit_status'] = start_ups_14['Exit_status'].astype(int)

In [12]:
#outliers treatment
start_ups_14['Estimated_Revenue_Range'] = start_ups_14['Estimated_Revenue_Range'].astype('int64')
start_ups_14['Estimated_Revenue_Range_z'] = stats.zscore(start_ups_14['Estimated_Revenue_Range'])

start_ups_14['Total_Funding_Amount'] = start_ups_14['Total_Funding_Amount'].astype('int64')
start_ups_14['Total_Funding_Amount_z'] = stats.zscore(start_ups_14['Total_Funding_Amount'])

start_ups_14['Number_of_Founders'] = start_ups_14['Number_of_Founders'].astype('int64')
start_ups_14['Number_of_Founders_z'] = stats.zscore(start_ups_14['Number_of_Founders'])

start_ups_14['Number_Funding_Rounds'] = start_ups_14['Number_Funding_Rounds'].astype('int64')
start_ups_14['Number_Funding_Rounds_z'] = stats.zscore(start_ups_14['Number_Funding_Rounds'])

start_ups_14['Number_of_Investors'] = start_ups_14['Number_of_Investors'].astype('int64')
start_ups_14['Number_of_Investors_z'] = stats.zscore(start_ups_14['Number_of_Investors'])

start_ups_14['Sector_Size'] = start_ups_14['Sector_Size'].astype('int64')
start_ups_14['Sector_Size_z'] = stats.zscore(start_ups_14['Sector_Size'])

start_ups_14['Number_of_Employees'] = start_ups_14['Number_of_Employees'].astype('int64')
start_ups_14['Number_of_Employees_z'] = stats.zscore(start_ups_14['Number_of_Employees'])

start_ups_14['Number_of_Apps'] = start_ups_14['Number_of_Apps'].astype('int64')
start_ups_14['Number_of_Apps_z'] = stats.zscore(start_ups_14['Number_of_Apps'])

start_ups_14['Visit_Duration'] = start_ups_14['Visit_Duration'].astype('int64')
start_ups_14['Visit_Duration_z'] = stats.zscore(start_ups_14['Visit_Duration'])

start_ups_14['Monthly_Visits'] = start_ups_14['Monthly_Visits'].astype('int64')
start_ups_14['Monthly_Visits_z'] = stats.zscore(start_ups_14['Monthly_Visits'])

start_ups_14['Bounce_Rate'] = start_ups_14['Bounce_Rate'].astype('int64')
start_ups_14['Bounce_Rate_z'] = stats.zscore(start_ups_14['Bounce_Rate'])

start_ups_14['Monthly_Visits_Growth'] = start_ups_14['Monthly_Visits_Growth'].astype('int64')
start_ups_14['Monthly_Visits_Growth_z'] = stats.zscore(start_ups_14['Monthly_Visits_Growth'])

In [13]:
# Replacing outliers
start_ups_14['Estimated_Revenue_Range'][(start_ups_14['Estimated_Revenue_Range_z'] >3) | (start_ups_14['Estimated_Revenue_Range_z'] <-3)] = statistics.median(start_ups_14['Estimated_Revenue_Range']) 

start_ups_14['Total_Funding_Amount'][(start_ups_14['Total_Funding_Amount_z'] >3) | (start_ups_14['Total_Funding_Amount_z'] <-3)] = statistics.median(start_ups_14['Total_Funding_Amount'])

start_ups_14['Number_of_Founders'][(start_ups_14['Number_of_Founders_z'] >3) | (start_ups_14['Number_of_Founders_z'] <-3)] = statistics.median(start_ups_14['Number_of_Founders'])

start_ups_14['Number_Funding_Rounds'][(start_ups_14['Number_Funding_Rounds_z'] >3) | (start_ups_14['Number_Funding_Rounds_z'] <-3)] = statistics.median(start_ups_14['Number_Funding_Rounds'])

start_ups_14['Number_of_Investors'][(start_ups_14['Number_of_Investors_z'] >3) | (start_ups_14['Number_of_Investors_z'] <-3)] = statistics.median(start_ups_14['Number_of_Investors'])

start_ups_14['Sector_Size'][(start_ups_14['Sector_Size_z'] >3) | (start_ups_14['Sector_Size_z'] <-3)] = statistics.median(start_ups_14['Sector_Size'])

start_ups_14['Number_of_Employees'][(start_ups_14['Number_of_Employees_z'] >3) | (start_ups_14['Number_of_Employees_z'] <-3)] = statistics.median(start_ups_14['Number_of_Employees'])

start_ups_14['Number_of_Apps'][(start_ups_14['Number_of_Apps_z'] >3) | (start_ups_14['Number_of_Apps_z'] <-3)] = statistics.median(start_ups_14['Number_of_Apps'])

start_ups_14['Visit_Duration'][(start_ups_14['Visit_Duration_z'] >3) | (start_ups_14['Visit_Duration_z'] <-3)] = statistics.median(start_ups_14['Visit_Duration'])

start_ups_14['Monthly_Visits'][(start_ups_14['Monthly_Visits_z'] >3) | (start_ups_14['Monthly_Visits_z'] <-3)] = statistics.median(start_ups_14['Monthly_Visits'])

start_ups_14['Bounce_Rate'][(start_ups_14['Bounce_Rate_z'] >3) | (start_ups_14['Bounce_Rate_z'] <-3)] = statistics.median(start_ups_14['Bounce_Rate'])

start_ups_14['Monthly_Visits_Growth'][(start_ups_14['Monthly_Visits_Growth_z'] >3) | (start_ups_14['Monthly_Visits_Growth_z'] <-3)] = statistics.median(start_ups_14['Monthly_Visits_Growth'])

In [14]:
start_ups_14 = start_ups_14[['Estimated_Revenue_Range', 'Total_Funding_Amount',
       'Headquarters_Location', 'Founded_Date', 'Number_of_Founders',
       'Funding_Stage', 'Number_Funding_Rounds', 'Number_of_Investors',
       'Industry_Groups', 'Sector_Size', 'Number_of_Employees',
       'Number_of_Apps', 'Visit_Duration', 'Bounce_Rate', 'Monthly_Visits',
       'Monthly_Visits_Growth', 'Exit_status']]

In [15]:
start_ups_14

,Estimated_Revenue_Range,Total_Funding_Amount,Headquarters_Location,Founded_Date,Number_of_Founders,Funding_Stage,Number_Funding_Rounds,Number_of_Investors,Industry_Groups,Sector_Size,Number_of_Employees,Number_of_Apps,Visit_Duration,Bounce_Rate,Monthly_Visits,Monthly_Visits_Growth,Exit_status
Organization_Name,,,,,,,,,,,,,,,,,
Nana,5500000,800000,Riyadh,2020,2,Early Stage,1,1,Food &Beverage,42000000000,376,1,561,58,80887,-30,0
Retailo Technologies,5500000,800000,Riyadh,2020,3,Mid stage,1,1,E-Commerce,13000000000,31,1,652,66,22598,1357,0
Lean Technologies,5500000,36500000,Riyadh,2019,3,Mid stage,1,1,Financial Services,13000000000,31,1,102,87,6677,-45,0
Sary,300000000,800000,Riyadh,2018,2,Late stage,1,1,E-Commerce,13000000000,376,1,178,40,25286,258,0
Tamara,5500000,800000,Riyadh,2020,3,Mid stage,1,1,Financial Services,13000000000,76,1,347,37,143987,74,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Riyadh Najed Schools,5500000,800000,Jeddah,2015,2,Mid stage,1,1,EduTech,4760000000,31,1,630,32,39028,34,0
Jeddah Filters,5500000,800000,Jeddah,2018,2,Mid stage,1,1,FMCG,42000000000,31,1,178,66,1606,-7,0
Ofoq Consulting,5500000,800000,Jeddah,2017,2,Late stage,1,1,IT,13000000000,31,1,178,66,1606,-7,0


In [16]:
start_ups_14 = pd.get_dummies (start_ups_14, columns=['Headquarters_Location', 'Funding_Stage', 'Industry_Groups'])

In [17]:
predictors = ['Estimated_Revenue_Range', 'Total_Funding_Amount', 'Founded_Date',
       'Number_of_Founders', 'Number_Funding_Rounds', 'Number_of_Investors',
       'Sector_Size', 'Number_of_Employees', 
       'Visit_Duration', 'Bounce_Rate', 'Monthly_Visits',
       'Monthly_Visits_Growth', 
       'Headquarters_Location_Al Qasim ',
       'Headquarters_Location_Ash Sharqiyah',
       'Headquarters_Location_Asir Province', 'Headquarters_Location_Hail',
       'Headquarters_Location_Jeddah', 'Headquarters_Location_Jizan',
       'Headquarters_Location_Riyadh', 'Funding_Stage_Early Stage',
       'Funding_Stage_Late stage', 'Funding_Stage_Mid stage',
       'Industry_Groups_Artificial Intelligence', 'Industry_Groups_E-Commerce',
       'Industry_Groups_EduTech', 'Industry_Groups_Energy',
       'Industry_Groups_FMCG', 'Industry_Groups_Financial Services',
       'Industry_Groups_Food &Beverage ', 'Industry_Groups_HealthTech',
       'Industry_Groups_Human Resources', 'Industry_Groups_IT',
       'Industry_Groups_Media&Entertainment', 'Industry_Groups_Real Estate',
       'Industry_Groups_Sales&Marketing', 'Industry_Groups_Travel&Toursim']

In [18]:
outcome = 'Exit_status'

X= start_ups_14[predictors]
y= start_ups_14[outcome]

# Scale the data
PredictorScaler=StandardScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)

# Generating the standardized values of X
X=PredictorScalerFit.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [19]:
# Oversample training data
sm = SMOTE(random_state = 2)
X_train, y_train = sm.fit_resample(X_train, y_train.ravel())

In [20]:
#XGBoost
XGB = XGBClassifier(max_depth=2, learning_rate=0.01, n_estimators=200, objective='binary:logistic', booster='gbtree')
XGB = XGB.fit(X_train, y_train)
XGB1 = XGB.predict(X_test)

In [21]:
# save preprocessing objects and RF algorithm

joblib.dump(dict_start, "./dict_start.joblib", compress=True)
joblib.dump(dict_start2, "./dict_start2.joblib", compress=True)
joblib.dump(dict_start3, "./dict_start3.joblib", compress=True)
joblib.dump(start_ups, "./start_ups.joblib", compress=True)
joblib.dump(start_ups_14, "./start_ups_14.joblib", compress=True)
#joblib.dump(encoders, "./encoders.joblib", compress=True)
joblib.dump(XGB, "./xgboost.joblib", compress=True)

['./xgboost.joblib']